<h3>Problem: As a PM, I write lots of blogs. How do I know if they will be received well by  readers?</h3>

<table>
    <tr>
        <td><img src="https://jayclouse.com/wp-content/uploads/2019/06/hacker_news.webp" height=300 width=300></img></td>
        <td><img src="https://miro.medium.com/max/852/1*wJ18DgYgtsscG63Sn56Oyw.png" height=300 width=300></img></td>
    </tr>
</table>

<h1>Background on Spark ML</h1>

DataFrame: This ML API uses DataFrame from Spark SQL as an ML dataset, which can hold a variety of data types. E.g., a DataFrame could have different columns storing text, feature vectors, true labels, and predictions.

Transformer: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.

Estimator: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.

Pipeline: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.

Parameter: All Transformers and Estimators now share a common API for specifying parameters.

In [2]:
from IPython.display import Image
Image(url='https://spark.apache.org/docs/3.0.0-preview/img/ml-Pipeline.png') 

<h2>Loading Hackernews Text From BigQuery</h2>

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
scala_minor_version = str(spark.sparkContext._jvm.scala.util.Properties.versionString().replace("version ","").split('.')[1])
spark = SparkSession.builder.config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2." + scala_minor_version + ":0.18.0") \
                                    .enableHiveSupport() \
                                    .getOrCreate()

In [5]:
df = spark.read \
  .format("bigquery") \
  .load("google.com:crosbie-test-project.demos.hackernewssample")

In [84]:
df.describe().show()

+-------+-----------------+--------------------+
|summary|            score|                text|
+-------+-----------------+--------------------+
|  count|             1000|                1000|
|   mean|            8.188|                null|
| stddev|44.01315500863138|                null|
|    min|                1|"Do you have a qu...|
|    max|             1081|هي شركة متخصصة في...|
+-------+-----------------+--------------------+



<h2>Prepare the data using Spark SQL<h2>
<h4>Create a random ID to distribute between test and training sets</h4>
<h4>Make the score a binary variable so we can run a logicistic regression model on it</h4>

In [86]:
df.registerTempTable("df")
from pyspark.sql import functions as F
df_full = spark.sql("select cast(round(rand() * 100) as int) as id, text, case when score > 10 THEN 1.0 else 0.0 end as label from df")

In [87]:
df_full.groupby('id').count().sort('count', ascending=False).show()

+---+-----+
| id|count|
+---+-----+
| 22|   17|
| 39|   16|
| 25|   15|
| 55|   15|
| 23|   15|
| 47|   15|
| 38|   15|
| 71|   14|
|  5|   14|
| 98|   14|
| 32|   14|
| 44|   13|
| 43|   13|
| 59|   13|
| 85|   13|
| 83|   13|
| 24|   13|
| 82|   13|
| 58|   13|
| 53|   13|
+---+-----+
only showing top 20 rows



<h4>Create our training and test sets</h4>

In [95]:
#use the above table to identify ~10% holdback for test
holdback = "(22,39,25,55,23,47,38,71,5,98)"

In [115]:
#create test set by dropping label 
df_test = df_full.where("id in {}".format(holdback))
df_test = df_test.drop("label")
rdd_test = df_test.rdd
test = rdd_test.map(tuple)
testing = spark.createDataFrame(test,["id", "text"])

In [99]:
#training data - Spark ML is expecting tuples so convert to RDD to map back to tuples (may not be required)
df_train = df_full.where("id not in {}".format(holdback))
rdd_train = df_train.rdd
train = rdd_train.map(tuple)
training = spark.createDataFrame(train,["id", "text", "label"])

In [114]:
#a little less than 10% of the trainig data is positively reviewed. Should be okay. 
training.where("label > 0").count()

91

<h2>Build our ML Pipeline</h2>

<h3>Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and lr.</h3>

In [44]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [116]:

tokenizer = Tokenizer(inputCol="text", outputCol="words")

hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")

lr = LogisticRegression(maxIter=10, regParam=0.001)

pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [117]:
# Fit the pipeline to hacker news articles
model = pipeline.fit(training)

<h3>Review model based on test set</h3>

In [119]:
# Make predictions on test documents and print columns of interest.
prediction = model.transform(testing)
selected = prediction.select("id", "text", "probability", "prediction").where("prediction > 0")
for row in selected.collect():
    rid, text, prob, prediction = row
    print("(%d, %s) --> prob=%s, prediction=%f" % (rid, text, str(prob), prediction))

(22, Other) --> prob=[0.3819756099050897,0.6180243900949102], prediction=1.000000
(55, While we may talk often about other pieces of the software development lifecycle, shell scripts don&#x27;t usually get to take center stage. Let&#x27;s share some creative little scripts we&#x27;ve come up with over the years!) --> prob=[0.4063985380451119,0.5936014619548882], prediction=1.000000


<h2>Use the model to decide which PM blog to use</h2>

In [122]:
my_blog = """
The life of a data scientist can be challenging. If you’re in this role, your job may involve anything from understanding the day-to-day business behind the data to keeping up with the latest machine learning academic research. With all that a data scientist must do to be effective, you shouldn’t have to worry about migrating data environments or dealing with processing limitations associated with working with raw data. 

Google Cloud’s Dataproc lets you run cloud-native Apache Spark and Hadoop clusters easily. This is especially helpful as data growth relocates data scientists and machine learning researchers from personal servers and laptops into distributed cluster environments like Apache Spark, which offers Python and R interfaces for data of any size. You can run open source data processing on Google Cloud, making Dataproc one of the fastest ways to extend your existing data analysis to cloud-sized datasets.  

We’re announcing the general availability of several new Dataproc features that will let you apply the open source tools, algorithms, and programming languages that you use today to large datasets. This can be done without having to manage clusters and computers. These new GA features make it possible for data scientists and analysts to build production systems based on personalized development environments. 
"""

pmm_blog = """
Dataproc makes open source data and analytics processing fast, easy, and more secure in the cloud.

New customers get $300 in free credits to spend on Dataproc or other Google Cloud products during the first 90 days. 

Go to console
Spin up an autoscaling cluster in 90 seconds on custom machines
Build fully managed Apache Spark, Apache Hadoop, Presto, and other OSS clusters
Only pay for the resources you use and lower the total cost of ownership of OSS
Encryption and unified security built into every cluster
Accelerate data science with purpose-built clusters
"""


boss_blog = """

In 2014, we made a decision to build our core data platform on Google Cloud Platform and one of the products which was critical for the decision was Google BigQuery. The scale at which it enabled us to perform analysis we knew would be critical in long run for our business. Today we have more than 200 unique users performing analysis on a monthly basis.

Once we started using Google BiqQuery at scale we soon realized our analysts needed better tooling around it. The key requests we started getting were

Ability to schedule jobs: Analysts needed to have ability to run queries at regular intervals to generate data and metrics.
Define workflow of queries: Basically analysts wanted to run multiple queries in a sequence and share data across them through temp tables.
Simplified data sharing: Finally it became clear teams needed to share this data generated with other systems. For example download it to leverage in R programs or send it to another system to process through Kafka.

"""

In [129]:
pm_blog_off = spark.createDataFrame([
    ('me', my_blog),
    ('pmm', pmm_blog),
    ('sudhir', boss_blog)
], ["id", "text"])

In [130]:
blog_prediction = model.transform(pm_blog_off)

In [131]:
blog_prediction.select("id","prediction").show()

+------+----------+
|    id|prediction|
+------+----------+
|    me|       0.0|
|   pmm|       1.0|
|sudhir|       0.0|
+------+----------+



<h2>Save our trained model to GCS</h2>

In [137]:
model.save("gs://crosbie-dev/blog-validation-model")